In [3]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertForSequenceClassification, BertTokenizer, TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [4]:
data = pd.read_csv('train_en.txt', sep='\t')

ParserError: Error tokenizing data. C error: EOF inside string starting at row 72104

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
sentences = data['Sentence'].values.tolist()


In [ ]:
encodings = tokenizer(sentences)

In [ ]:
encodings['attention_mask']

In [ ]:
labels = [1 if label=='toxic' else 0 for label in data['Style'].values]


In [ ]:
class ToxicDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
dataset = ToxicDataset(encodings, labels)


In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2
)

In [ ]:
training_args = TrainingArguments(output_dir='toxic_bert', report_to='none', learning_rate=0.001,
                                  num_train_epochs=5, weight_decay=0.005)

In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=dataset, tokenizer=tokenizer)

In [ ]:
trainer.train()